In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Univ.AI Hackathon/Training Data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Univ.AI Hackathon/Test Data.csv')

In [ ]:
train.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [ ]:
train.shape

(252000, 13)

In [ ]:
dummies = pd.get_dummies(train['house_ownership'])
#dummies.drop('norent_noown', axis=1, inplace=True)

In [ ]:
train = train.merge(dummies, left_index=True, right_index=True)



train.shape

(252000, 16)

In [ ]:
dummie = pd.get_dummies(test_data['house_ownership'])

#dummie.drop('norent_noown', axis=1, inplace=True)

In [ ]:
test = test_data.merge(dummie, left_index=True, right_index=True)


test.shape

(28000, 15)

In [ ]:
test.head()

,id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,norent_noown,owned,rented
0,1,7393090,59,19,single,rented,no,Geologist,Malda,West Bengal,4,13,0,0,1
1,2,1215004,25,5,single,rented,no,Firefighter,Jalna,Maharashtra,5,10,0,0,1
2,3,8901342,50,12,single,rented,no,Lawyer,Thane,Maharashtra,9,14,0,0,1
3,4,1944421,49,9,married,rented,yes,Analyst,Latur,Maharashtra,3,12,0,0,1
4,5,13429,25,18,single,rented,yes,Comedian,Berhampore,West Bengal,13,11,0,0,1


In [ ]:
train["married"]=pd.factorize(train.married)[0]
train["car_ownership"]=pd.factorize(train.car_ownership)[0]
#train["house_ownership"]=pd.factorize(train.house_ownership)[0]

In [ ]:
test["married"]=pd.factorize(test_data.married)[0]
test["car_ownership"]=pd.factorize(test_data.car_ownership)[0]
#test["house_ownership"]=pd.factorize(test_data.house_ownership)[0]

In [ ]:
train['car_ownership'].value_counts()

0    176000
1     76000
Name: car_ownership, dtype: int64

In [ ]:
train.drop('city', inplace=True, axis=1)
train.drop('experience', inplace=True, axis=1)
train.drop('profession', inplace=True, axis=1)
train.drop('state', inplace=True, axis=1)
train.drop('Id', inplace=True, axis=1)
train.drop('house_ownership', inplace=True, axis=1)

In [ ]:
test.drop('city', inplace=True, axis=1)
test.drop('experience', inplace=True, axis=1)
test.drop('profession', inplace=True, axis=1)
test.drop('state', inplace=True, axis=1)
test.drop('id', inplace=True, axis=1)
test.drop('house_ownership', inplace=True, axis=1)

In [ ]:
train.shape, test.shape

((252000, 10), (28000, 9))

In [ ]:
x_train=train.drop('risk_flag', axis=1)
y_train=train['risk_flag']

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(x_train, y_train, random_state=201, test_size=0.2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = x_minmax.transform(X_train)
X_test_scaled = x_minmax.transform(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

x_ss = StandardScaler().fit(X_train)

X_train_ss = x_ss.transform(X_train)
X_test_ss = x_ss.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential([
                                 keras.layers.Dense(512, input_dim=9, activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(256, activation='relu'),
                                 keras.layers.Dense(128, activation='relu'),
                                 keras.layers.Dense(64, activation='relu'),
                                 keras.layers.Dense(32, activation='relu'),
                                 keras.layers.Dense(1, activation='sigmoid') #softmax
])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=5, verbose=1, batch_size=256, validation_split=0.6)

predsNN = model.predict_proba(X_test_scaled)

roc_auc_score(X_test, predsNN[:, 1])

In [ ]:
from xgboost.sklearn import XGBClassifier

xgb = XGBClassifier(random_state=21)

xgb.fit(X_train_scaled, Y_train)

predictions = xgb.predict_proba(X_test_scaled)
# predictions2 = xgb.predict_proba(X_test_ss)

roc_auc_score(Y_test, predictions[:,1])  

0.6217073075077493

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
NB = GaussianNB()
NBtuned = CalibratedClassifierCV(NB, cv=5, method='sigmoid')

NBtuned.fit(X_train_scaled, Y_train)

NBpredsTrain = NBtuned.predict_proba(X_train_scaled)
NBpreds = NBtuned.predict_proba(X_test_scaled)


roc_auc_score(Y_train, NBpredsTrain[:, 1]), roc_auc_score(Y_test, NBpreds[:, 1])

(0.5499072197870953, 0.5523455855404862)

In [ ]:
from xgboost import XGBClassifier

xg = XGBClassifier(objective='binary:logistic', random_state=2021)
xg.fit(X_train_scaled, Y_train)

xgpredsTrain = xg.predict_proba(X_train_scaled) 
xgpreds = xg.predict_proba(X_test_scaled)
print(roc_auc_score(Y_train, xgpredsTrain[:, 1]))
roc_auc_score(Y_test, xgpreds[:, 1])

0.6378740175179418


0.6198947388371845

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=21)

DT.fit(X_train_scaled, Y_train)

DTpreds = DT.predict_proba(X_test_scaled)
trainDTpreds = DT.predict_proba(X_train_scaled)
DTpredsp = DT.predict(X_test_scaled)

roc_auc_score(Y_test, DTpreds[:,1]), roc_auc_score(Y_train, trainDTpreds[:,1]), roc_auc_score(Y_test, DTpredsp)

(0.8474339202568182, 0.9780481761999213, 0.743826735514007)

In [ ]:
# DT.get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

poly_svm = Pipeline((('poly', PolynomialFeatures(degree=5)), ('svm', SVC(kernel='poly', C=5))))
poly_svm.fit(X_train_scaled, Y_train)

svmpreds = poly_svm.predict_proba(X_test_scaled)

print(roc_auc_score(Y_test, svmpreds))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state=34)
RF.fit(X_train_scaled, Y_train)

RFpreds = RF.predict_proba(X_test_scaled)
RFpredsTrain = RF.predict_proba(X_train_scaled)
print(roc_auc_score(Y_train, RFpredsTrain[:, 1]))
print('auc_roc_score ', roc_auc_score(Y_test, RFpreds[:, 1]))

0.9757910216078367
auc_roc_score  0.9346048077890875


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

classifier = AdaBoostClassifier()

classifier.fit(X_train_scaled, Y_train)
Adapreds=classifier.predict_proba(X_test_scaled)
roc_auc_score(Y_test, Adapreds[:, 1])

0.5703573282751686

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_scaled, Y_train)
preds = lr.predict_proba(X_test_scaled)
roc_auc_score(Y_test, preds[:,1])

0.5463910591189116

In [ ]:
Test_scaled = x_minmax.transform(test)

In [ ]:
predictions = RF.predict(Test_scaled)

In [ ]:
test_data['risk_flag']=predictions


In [ ]:
predictions = test_data[['id', 'risk_flag']]
predictions = np.array(predictions, dtype=np.int32)

In [ ]:
np.savetxt('/content/drive/MyDrive/Colab Notebooks/Univ.AI Hackathon/predictionsLast.csv',predictions, delimiter=',')